In [1]:
from SFS.src.py.utils import *
from numpy.fft import fft2, fft, ifft, fftfreq, fftshift, ifftshift, rfftfreq, rfft2
from IPython.display import HTML

In [2]:
def get_K(field, con):
    r, u = con["r"], con["u"]
    return - ( r + u * field**2 ) * field

def get_qw(param, mask=False):
    T, N, M, L, TIME = param
    q = (fftfreq(N, L / (2 * np.pi * N)))
    w = (fftfreq(M, TIME / (2 * np.pi * M)))
    q, w = np.meshgrid(q, w) 
    q, w = fftshift(q), fftshift(w)
    if mask:
        mask = q==0
        w = np.ma.array(w, mask=mask)
        q = np.ma.array(q, mask=mask)
    return q, w

In [3]:
def get_corr(seed, folder, start=0):
    run_folder = folder + "{m}/".format(m = seed)
    X, d, N, L, T, dt, con = get_para(run_folder)

    fn = "varphi"
    field = get_field(run_folder, fn)
    samples = len(field)
    M = samples - start
    times = get_time(run_folder)[start:]
    field = field[start:]
    TIME = times[-1] - times[0]

    param = T, N, M, L, TIME

    q, w = get_qw(param)

    # win = np.hanning(M) # hanning window
    win = np.ones(M)

    field = field * win[:, None]

    phiqw = fft2(field) * (L/N) * (TIME/M)
    phiqw = fftshift(phiqw)

    Cqw = phiqw * np.conj(phiqw) / L / TIME

    K = get_K(field, con)
    Kqw = fft2(K) * (L/N) * (TIME/M)
    Kqw = fftshift(Kqw) - q**2 * phiqw

    Xqw = 1 / (2*T) * (phiqw * np.conj(- 1j * w * phiqw - Kqw ) ) / L / TIME

    mask = Cqw < 1e-20

    w = np.ma.array(w, mask=mask)
    q = np.ma.array(q, mask=mask)
    Cqw = np.ma.array(Cqw, mask=mask)
    Xqw = np.ma.array(Xqw, mask=mask)
    
    return (q, w), Cqw, Xqw, con, param

In [4]:
def plot(C, X, q, w, param, wlim=None, qlim=None, size=12):
    T, N, M, L, TIME = param
    
    fig, ax = plt.subplots(1, 3, figsize=(size, size*0.25), sharey=True)

    p = ax[0].pcolor(w, q, C, norm=colors.LogNorm(np.min(C), np.max(C)))
    ax[0].set_xlabel("$\\omega$")
    ax[0].set_ylabel("$q$")
    ax[0].set_title("$C(q,\\omega)$", fontsize=20)
    cbar = fig.colorbar(p, ax=ax[0])

    ticks = cbar.get_ticks()
    norm = (ticks <= np.max(C)) & (ticks >= np.min(C))
    ticks = ticks[norm]
    lticks = np.array(np.log10(ticks), dtype=int)
    cbar.set_ticks(ticks=ticks,labels=lticks)
    ax[0].text(14, 2.2,"$\\text{log}_{10} C$")

    ImX = np.imag(X)
    lim = np.max(np.abs(ImX)) * 1.14
    p = ax[1].pcolor(w, q, ImX, vmin=-lim, vmax=lim)
    ax[1].set_xlabel("$\\omega$")

    ax[1].set_title("$\\mathrm{Im} \\chi(q,\\omega)$", fontsize=20)
    cbar = fig.colorbar(p, ax=ax[1])
    cbar.formatter.set_powerlimits((0,0)) 
    cbar.formatter.set_useMathText(True)
    
    if not wlim is None:
        ax[0].set_xlim(-wlim, wlim)
        ax[1].set_xlim(-wlim, wlim)
    if not qlim is None:
        ax[0].set_ylim(-qlim, qlim)
        ax[1].set_ylim(-qlim, qlim)


    DFDT = (2 * ImX)-(w * C / T)

    lim = np.max(np.abs(DFDT))
    err = np.mean(np.abs(DFDT))

    p = plt.pcolor(w, q, DFDT, cmap=cm.bwr, norm=colors.Normalize(-lim, lim))
    ax[2].set_xlabel("$\\omega$")
    ax[2].set_title("$2 \\mathrm{Im} \\chi - \\omega \\beta  C$", fontsize=20)
    cbar = fig.colorbar(p, ax=ax[2]) 
    cbar.formatter.set_powerlimits((0,0)) 
    cbar.formatter.set_useMathText(True)

    return fig, ax

In [5]:
def get_CX(num, folder):
    (q, w), C, X, con, param = get_corr(1, folder)
    count = 1

    for i in range(1, num):
        Ci, Xi = get_corr(i+1, folder)[1:3]
        C += Ci
        X += Xi
        count += 1
        
    C = C / count
    X = X / count

    if np.max(np.abs(np.imag(C))) < 1e-10: C = np.real(C)
    return (q, w), C, X, con, param

In [7]:
number = 4
folder = "data/SETD1/{n}/".format(n=number)
# folder = "/data.lmp/mjohnsrud/SETD1/{n}/".format(n=number)
num = count_files(folder)
print(num)
num = 2**6

0


In [ ]:
(q, w), C, X, con, param = get_CX(num, folder)
fig, ax = plot(C, X, q, w, param, size=17)
T, N, M, L, TIME = param
title = "$N={num}, r={r}, u={u}, T={T}$".format(num=num,r=con["r"], u=con["u"], T=T)
print(title)
plt.savefig("plot/fig/SETD_paper/fig_fdt.png", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

T, N, M, L, TIME = param
DFDT = (2 * np.imag(X))/(w * C / T) - 1
DFDT[w==0] = 0

lim = np.max(np.abs(DFDT))
err = np.mean(np.abs(DFDT))

p = plt.pcolor(w, q, np.abs(w*C/T), cmap=cm.viridis)
fig.colorbar(p, label="$\\omega \\beta  C$", ax=ax)
ax.set_xlabel("$\\omega$")
ax.set_ylabel("$q$")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

DFDT = (2 * np.imag(X))-(w * C / T)

lim = np.max(np.abs(DFDT))
err = np.mean(np.abs(DFDT))

p = plt.pcolor(w, q, DFDT, cmap=cm.bwr, norm=colors.Normalize(-lim, lim))
fig.colorbar(p, label="$2 \\mathrm{Im} \\chi - \\omega \\beta  C$", ax=ax)
ax.set_title("$N={num}, \\bar\\Delta={err:.8f}$".format(num=num, err=err))
ax.set_xlabel("$\\omega$")
ax.set_ylabel("$q$")
plt.show()

In [ ]:
NN = 6
rows = 2
errs = []
errs2 = []
nums = [2**n for n in range(1,NN+1,1)]
fig, ax = plt.subplots(rows, (len(nums)-1)//rows+1, figsize=(12, 4), sharey=True, sharex=True)
ax = np.atleast_2d(ax)
axf = ax.flatten()

for i, num in enumerate(nums):
    (q, w), Ci, Xi, con, param = get_CX(num, folder)

    DFDT = (2 * np.imag(Xi))- (w * Ci / T)

    err = np.mean(np.abs(DFDT))
    errs.append(err)

    if i == 0: lim = np.max(np.abs(DFDT))
    p = axf[i].pcolor(w, q, DFDT, cmap=cm.bwr, norm=colors.Normalize(-lim, lim))
    axf[i].set_title("$N={num}, \\bar \\Delta={err:.5f}$".format(num=num, err=err), y=.82, fontsize=9)

    err2 = np.mean(DFDT)
    errs2.append(err2)

ax[-1,0].set_xlabel("$\\omega$")
ax[-1,0].set_ylabel("$q$")

cbar = fig.colorbar(p, label="$\\Delta = 2 \\mathrm{Im} \\chi - \\omega \\beta  C$", ax=ax)
[t.set_fontsize(14) for t in cbar.ax.get_yticklabels()]


plt.show()

In [ ]:
nums = np.array(nums) * 1.
fig, ax = plt.subplots(figsize=(5,4))
ax.loglog(nums, (nums/nums[-1])**(-1/2)*errs[-1], '--', label='$\\propto \\frac{1}{\\sqrt{s}}$')
ax.loglog(nums, errs, 'ko', label="$ \\sum_{ab} \\,\\frac{|\\Delta_{ab}(s)| }{N M}$")
ax.set_xlabel("$\\text{log}_{10} s$")
ax.set_ylabel("$\\text{log}_{10}  \\bar \\Delta$")

ax.tick_params(direction='in')
ax.tick_params(direction='in', which="minor")

yt = ax.get_yticks()
ax.set_yticklabels(np.array(np.log10(yt), dtype=int))
ax.set_yticklabels([], minor=True)

xt = ax.get_xticks()
ax.set_xticklabels(np.array(np.log10(xt), dtype=int))
ax.set_xticklabels([], minor=True)

ax.legend(fontsize=18)
plt.savefig("plot/fig/SETD_paper/fig_fdt2.pdf", bbox_inches="tight")
plt.show()

In [ ]:

nums = np.array(nums) * 1.
fig, ax = plt.subplots(figsize=(5,4))
ax.loglog(nums, np.abs(errs2), 'ko', label="$ \\sum_{ab} \\,\\frac{\\Delta_{ab}(s) }{N M}$")

ax.tick_params(direction='in')
ax.tick_params(direction='in', which="minor")

yt = ax.get_yticks()
ax.set_yticklabels([], minor=True)

xt = ax.get_xticks()
ax.set_xticklabels([])
ax.set_xticklabels([], minor=True)

ax.legend(fontsize=28)
plt.savefig("plot/fig/SETD_paper/fig_fdt3.pdf", bbox_inches="tight")
plt.show()